In [1]:
import sqlite3
import pandas as pd

In [2]:
db = sqlite3.connect('sqlite_db_pythonsqlite.db')

* PART 1: PHPMyAdmin


Q1: Some of the facilities charge a fee to members, but some do not.Write a SQL query to produce a list of the names of the facilities that do.

In [3]:
pd.read_sql_query("SELECT DISTINCT(name) FROM Facilities WHERE membercost != 0", db)

,name
0,Tennis Court 1
1,Tennis Court 2
2,Massage Room 1
3,Massage Room 2
4,Squash Court


Q2: How many facilities do not charge a fee to members?


In [4]:
pd.read_sql_query("SELECT COUNT(name) FROM Facilities WHERE membercost != 0", db)

,COUNT(name)
0,5


Q3: Write an SQL query to show a list of facilities that charge a fee to members,
where the fee is less than 20% of the facility's monthly maintenance cost.
Return the facid, facility name, member cost, and monthly maintenance of the
facilities in question.

In [5]:
pd.read_sql_query('''SELECT facid, name, membercost, monthlymaintenance 
FROM Facilities 
WHERE membercost <= 0.2*monthlymaintenance''', db)

,facid,name,membercost,monthlymaintenance
0,0,Tennis Court 1,5.0,200
1,1,Tennis Court 2,5.0,200
2,2,Badminton Court,0.0,50
3,3,Table Tennis,0.0,10
4,4,Massage Room 1,9.9,3000
5,5,Massage Room 2,9.9,3000
6,6,Squash Court,3.5,80
7,7,Snooker Table,0.0,15
8,8,Pool Table,0.0,15


Q4: Write an SQL query to retrieve the details of facilities with ID 1 and 5.
Try writing the query without using the OR operator. 

In [6]:
pd.read_sql_query('''SELECT * FROM Facilities WHERE facid IN (1, 5)''', db)

,facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,1,Tennis Court 2,5.0,25,8000,200
1,5,Massage Room 2,9.9,80,4000,3000


Q5: Produce a list of facilities, with each labelled as
'cheap' or 'expensive', depending on if their monthly maintenance cost is
more than $100. Return the name and monthly maintenance of the facilities
in question.

In [7]:
pd.read_sql_query('''SELECT name, monthlymaintenance,
CASE WHEN monthlymaintenance > 100 THEN 'expensive'
    ELSE 'cheap' END AS monthly_maintenance_cost
FROM Facilities
''', db)

,name,monthlymaintenance,monthly_maintenance_cost
0,Tennis Court 1,200,expensive
1,Tennis Court 2,200,expensive
2,Badminton Court,50,cheap
3,Table Tennis,10,cheap
4,Massage Room 1,3000,expensive
5,Massage Room 2,3000,expensive
6,Squash Court,80,cheap
7,Snooker Table,15,cheap
8,Pool Table,15,cheap


Q6: You'd like to get the first and last name of the last member(s)
who signed up. Try not to use the LIMIT clause for your solution.

In [8]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db') 
cur = con.cursor() 
last_row = cur.execute('SELECT surname, firstname, joindate FROM Members ORDER BY joindate').fetchall()[-1]
last_row

('Smith', 'Darren', '2012-09-26 18:08:45')

In [9]:
con = sqlite3.connect('sqlite_db_pythonsqlite.db')
rs = con.execute('SELECT surname, firstname, joindate FROM Members ORDER BY joindate DESC')
df = pd.DataFrame(rs.fetchall())
print(df.head(1))
con.close()

       0       1                    2
0  Smith  Darren  2012-09-26 18:08:45


Q7: Produce a list of all members who have used a tennis court.
Include in your output the name of the court, and the name of the member
formatted as a single column. Ensure no duplicate data, and order by
the member name.

In [10]:
tennis_members = pd.read_sql_query('''SELECT DISTINCT(firstname || " " || surname) AS member_name, name
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid 
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE name IN ('Tennis Court 1', 'Tennis Court 2')
ORDER BY member_name
''', db)

In [11]:
tennis_members

,member_name,name
0,Anne Baker,Tennis Court 1
1,Anne Baker,Tennis Court 2
2,Burton Tracy,Tennis Court 2
3,Burton Tracy,Tennis Court 1
4,Charles Owen,Tennis Court 1
5,Charles Owen,Tennis Court 2
6,Darren Smith,Tennis Court 2
7,David Farrell,Tennis Court 1
8,David Farrell,Tennis Court 2
9,David Jones,Tennis Court 2


Q8: Produce a list of bookings on the day of 2012-09-14 which
will cost the member (or guest) more than $30. Remember that guests have
different costs to members (the listed costs are per half-hour 'slot'), and
the guest user's ID is always 0. Include in your output the name of the
facility, the name of the member formatted as a single column, and the cost.
Order by descending cost, and do not use any subqueries.

In [12]:
pd.read_sql_query("""SELECT f.name, 
firstname || " " || surname AS member_name, 
CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS Cost
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid 
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%' AND cost > 30.0""", db)

,name,member_name,Cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


Q9: This time, produce the same result as in Q8, but using a subquery.

In [13]:
pd.read_sql_query("""SELECT * FROM (SELECT f.name, 
firstname || " " || surname AS member_name, 
CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS Cost
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid 
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE starttime LIKE '2012-09-14%') WHERE cost > 30""", db)

,name,member_name,Cost
0,Tennis Court 1,GUEST GUEST,75.0
1,Tennis Court 1,GUEST GUEST,75.0
2,Tennis Court 2,GUEST GUEST,75.0
3,Tennis Court 2,GUEST GUEST,150.0
4,Massage Room 1,GUEST GUEST,160.0
5,Massage Room 1,GUEST GUEST,160.0
6,Massage Room 1,Jemima Farrell,39.6
7,Massage Room 1,GUEST GUEST,160.0
8,Massage Room 2,GUEST GUEST,320.0
9,Squash Court,GUEST GUEST,70.0


* PART 2: SQLite

Export the country club data from PHPMyAdmin, and connect to a local SQLite instance from Jupyter notebook 
for the following questions.  


Q10: Produce a list of facilities with a total revenue less than 1000.
The output of facility name and total revenue, sorted by revenue. Remember
that there's a different cost for guests and members!

In [14]:
pd.read_sql_query("""SELECT name, SUM(Cost) - initialoutlay - monthlymaintenance*(strftime('%m',MAX(starttime)) - strftime('%m',MIN(starttime))) AS revenue
FROM(SELECT *, 
CASE WHEN m.memid = 0 THEN guestcost*slots ELSE membercost*slots END AS Cost
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid 
LEFT JOIN Members AS m
ON b.memid = m.memid)
GROUP BY name
HAVING revenue < 1000
ORDER BY revenue
""", db)


,name,revenue
0,Badminton Court,-2193.5
1,Snooker Table,-240.0
2,Pool Table,-160.0
3,Table Tennis,-160.0


Q11: Produce a report of 'members' and 'who recommended them' in alphabetic surname, firstname order

In [16]:
pd.read_sql_query(""" SELECT m1.firstname || " " || m1.surname AS member_name,  m2.firstname || " " || m2.surname AS recommender 
FROM Members AS m1
INNER JOIN Members AS m2
ON m1.recommendedby = m2.memid 
ORDER BY m1.surname, m1.firstname, m2.surname, m2.firstname
""", db)

,member_name,recommender
0,Florence Bader,Ponder Stibbons
1,Anne Baker,Ponder Stibbons
2,Timothy Baker,Jemima Farrell
3,Tim Boothe,Tim Rownam
4,Gerald Butters,Darren Smith
5,Joan Coplin,Timothy Baker
6,Erica Crumpet,Tracy Smith
7,Nancy Dare,Janice Joplette
8,Matthew Genting,Gerald Butters
9,John Hunt,Millicent Purview


Q12: Find the facilities with their usage by member, but not guests

In [22]:
pd.read_sql_query("""SELECT m.firstname || " " || m.surname AS member_name, name, SUM(slots) AS usage
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE b.memid NOT IN (0)
GROUP BY member_name, name
""", db)

,member_name,name,usage
0,Anna Mackenzie,Badminton Court,96
1,Anna Mackenzie,Massage Room 1,2
2,Anna Mackenzie,Pool Table,83
3,Anna Mackenzie,Snooker Table,14
4,Anna Mackenzie,Squash Court,4
...,...,...,...
197,Tracy Smith,Snooker Table,90
198,Tracy Smith,Squash Court,12
199,Tracy Smith,Table Tennis,56
200,Tracy Smith,Tennis Court 1,93


Q13: Find the facilities usage (시설 이용도) by month, but not guests

In [37]:
pd.read_sql_query("""SELECT strftime('%m', starttime) AS month, name, SUM(slots) AS usage
FROM Bookings AS b
LEFT JOIN Facilities AS f
ON b.facid = f.facid
LEFT JOIN Members AS m
ON b.memid = m.memid
WHERE b.memid NOT IN (0)
GROUP BY month, name
""", db)

,month,name,usage
0,07,Badminton Court,165
1,07,Massage Room 1,166
2,07,Massage Room 2,8
3,07,Pool Table,110
4,07,Snooker Table,140
5,07,Squash Court,50
6,07,Table Tennis,98
7,07,Tennis Court 1,201
8,07,Tennis Court 2,123
9,08,Badminton Court,414
